In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from collections import Counter

from matplotlib import pyplot as plt
from matplotlib import style

from scipy.sparse.linalg import eigs, svds

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [3]:
import csv
import datetime
import gc
import math
import networkx as nx
import numpy as np
import os
import pandas as pd
import pdb
import pickle
import random
import seaborn as sns
import time
import xgboost as xgb

__Reading data__

In [4]:
if os.path.isfile(path='data/after_eda/train_pos_after_eda.csv'):
    train_graph = nx.read_edgelist(path='data/after_eda/train_pos_after_eda.csv',
                                   delimiter=',',
                                   create_using=nx.DiGraph(),
                                   nodetype=int)
    print(nx.info(train_graph))
else:
    print("Please run the FB-Friend-Recommendation-EDA.ipynb file.")

DiGraph with 1780722 nodes and 7550015 edges


__Similarity measures__

__Jaccard index__

Link: http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

In [5]:
def jaccard_index_for_followees(a, b):
    """
    This function calculates the jaccard index for followees.
    """
    try:
        X = set(train_graph.successors(a))
        Y = set(train_graph.successors(b))
        if len(X) == 0  or len(Y) == 0:
            return 0
        sim = len(X.intersection(Y)) / len(X.union(Y))
        return sim
    except:
        return 0

In [6]:
print(jaccard_index_for_followees(273084, 1505602))

0.0


In [7]:
print(jaccard_index_for_followees(1, 3))

0.0


In [8]:
def jaccard_index_for_followers(a, b):
    """
    This function calculates the jaccard index for followers.
    """
    try:
        X = set(train_graph.predecessors(a))
        Y = set(train_graph.predecessors(b))
        if len(X) == 0  or len(Y) == 0:
            return 0
        sim = len(X.intersection(Y)) / len(X.union(Y))
        return sim
    except:
        return 0

In [9]:
print(jaccard_index_for_followers(273084, 1505602))

0.0


In [10]:
print(jaccard_index_for_followers(1, 3))

0.0


__Cosine distance (Otsuka–Ochiai coefficient)__

Link: https://en.wikipedia.org/wiki/Cosine_similarity#Otsuka%E2%80%93Ochiai_coefficient

\begin{equation}
d = \frac{|X \cap Y|}{\sqrt{|X| \times |Y|}}
\end{equation}

In [11]:
def cosine_distance_for_followees(a, b):
    """
    This function calculates the cosine distance for followees.
    """
    try:
        X = set(train_graph.successors(a))
        Y = set(train_graph.successors(b))
        if len(X) == 0  | len(Y) == 0:
            return 0
        sim = len(X.intersection(Y)) / math.sqrt(len(X) * len(Y))
        return sim
    except:
        return 0

In [12]:
print(cosine_distance_for_followees(273084, 1505602))

0.0


In [13]:
print(cosine_distance_for_followees(1, 2))

0.0


In [14]:
def cosine_distance_for_followers(a, b):
    """
    This function calculates the cosine distance for followers.
    """
    try:
        X = set(train_graph.predecessors(a))
        Y = set(train_graph.predecessors(b))
        if len(X) == 0  | len(Y) == 0:
            return 0
        sim = len(X.intersection(Y)) / math.sqrt(len(X) * len(Y))
        return sim
    except:
        return 0

In [15]:
print(cosine_distance_for_followers(273084, 1505602))

0.0


In [16]:
print(cosine_distance_for_followers(1, 2))

0.0


__Ranking measures__

Link: https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/1024px-PageRanks-Example.svg.png)

Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. (The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.

__Page ranking__

Link: https://en.wikipedia.org/wiki/PageRank

In [17]:
if not os.path.isfile(path='data/fea_sample/page_rank.p'):
    pr = nx.pagerank(G=train_graph, alpha=0.85)
    pickle.dump(obj=pr, file=open(file='data/fea_sample/page_rank.p', mode='wb'))
else:
    pr = pickle.load(file=open(file='data/fea_sample/page_rank.p', mode='rb'))

In [18]:
print("Min: {} --> {}".format(min(pr, key=pr.get), pr[min(pr, key=pr.get)]))
print("Max: {} --> {}".format(max(pr, key=pr.get), pr[max(pr, key=pr.get)]))
print("Mean: {}".format(float(sum(pr.values()) / len(pr))))

Min: 527014 --> 1.6556497245737814e-07
Max: 289584 --> 2.7098251341935827e-05
Mean: 5.615699699389075e-07


In [19]:
mean_pr = float(sum(pr.values()) / len(pr))
print(mean_pr)

5.615699699389075e-07


__Other graph features__

__Shortest path__

Getting Shortest path between twoo nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path.

Link: https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html

In [20]:
def compute_shortest_path_length(a, b):
    """
    This function computes the shortest path between nodes.
    """
    p = -1
    try:
        if train_graph.has_edge(u=a, v=b):
            train_graph.remove_edge(u=a, v=b)
            p = nx.shortest_path_length(G=train_graph, source=a, target=b)
            train_graph.add_edge(u_of_edge=a, v_of_edge=b)
        else:
            p = nx.shortest_path_length(G=train_graph, source=a, target=b)
        return p
    except:
        return -1

In [21]:
print(compute_shortest_path_length(a=77697, b=826021))

10


In [22]:
print(compute_shortest_path_length(a=669354, b=1635354))

-1


__Checking for the same community__

In [23]:
wcc = list(nx.weakly_connected_components(G=train_graph))

In [24]:
def belongs_to_same_wcc(a, b):
    """
    This function checks the belongingness of node_a and node_b in same community.
    """
    index = []
    if train_graph.has_edge(u=b, v=a):
        return 1
    if train_graph.has_edge(u=a, v=b):
        for i in wcc:
            if a in i:
                index = i
                break
        if (b in index):
            train_graph.remove_edge(u=a, v=b)
            if compute_shortest_path_length(a=a, b=b) == -1:
                train_graph.add_edge(u_of_edge=a, v_of_edge=b)
                return 0
            else:
                train_graph.add_edge(u_of_edge=a, v_of_edge=b)
                return 1
        else:
            return 0
    else:
        for i in wcc:
            if a in i:
                index = i
                break
        if(b in index):
            return 1
        else:
            return 0

In [25]:
print(belongs_to_same_wcc(a=861, b=1659750))

0


In [26]:
print(belongs_to_same_wcc(a=669354, b=1635354))

0


__Adamic/Adar index__

Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [29]:
def compute_adar_index(a, b):
    """
    This function computes Adar index.
    """
    sum_ = 0
    try:
        n = list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n) != 0:
            for i in n:
                sum_ = sum_ + (1 / np.log10(len(list(set(train_graph.predecessors(i))))))
            return sum_
        else:
            return 0
    except:
        return 0

In [30]:
print(compute_adar_index(a=1, b=189226))

0


In [31]:
print(compute_adar_index(a=669354, b=1635354))

0


__Is person following back?__

In [32]:
def follows_back(a, b):
    """
    Boolean function.
    """
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [33]:
print(follows_back(a=1, b=189226))

1


In [34]:
print(follows_back(a=669354, b=1635354))

0


__Katz centrality__

Katz centrality computes the centrality for a node based on the centrality of its neighbors. It is a generalization of the eigenvector centrality. The Katz centrality for node `i` is, $$x_i = \alpha \sum_{j} A_{ij} x_j + \beta$$ where `A` is the adjacency matrix of the graph G with eigenvalues $\lambda$. The parameter $\beta$ controls the initial centrality and $\alpha < \frac{1}{\lambda_{max}}$.

In [35]:
if not os.path.isfile(path='data/fea_sample/katz.p'):
    katz = nx.katz.katz_centrality(G=train_graph, alpha=0.005, beta=1)
    pickle.dump(obj=katz, file=open(file='data/fea_sample/katz.p', mode='wb'))
else:
    katz = pickle.load(file=open(file='data/fea_sample/katz.p', mode='rb'))

In [36]:
print('Min: {}'.format(katz[min(katz, key=katz.get)]))
print('Max: {}'.format(katz[max(katz, key=katz.get)]))
print('Mean: {}'.format(float(sum(katz.values())) / len(katz)))

Min: 0.0007313532484065916
Max: 0.003394554981699122
Mean: 0.0007483800935562018


In [37]:
mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

0.0007483800935562018


__Hits score__

The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.

Link: https://en.wikipedia.org/wiki/HITS_algorithm

In [38]:
if not os.path.isfile(path='data/fea_sample/hits.p'):
    hits = nx.hits(G=train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(obj=hits, file=open(file='data/fea_sample/hits.p', mode='wb'))
else:
    hits = pickle.load(file=open(file='data/fea_sample/hits.p', mode='rb'))

In [39]:
print('Min: {}'.format(hits[0][min(hits[0], key=hits[0].get)]))
print('Max: {}'.format(hits[0][max(hits[0], key=hits[0].get)]))
print('Mean: {}'.format(float(sum(hits[0].values())) / len(hits[0])))

Min: 0.0
Max: 0.004868653378780953
Mean: 5.615699699344123e-07
